In [9]:
import numpy as np
import sys
sys.path.append("..")
from solvers.ga.ga_solver import GeneticAlgorithmSolver
from problem.sudoku_manager import Sudoku, generate_boards

In [10]:
sud = Sudoku('medium', 34)

In [11]:
new_solver = GeneticAlgorithmSolver(pop_size=500,  pc=0.8, pm=0.15, max_epoch=700, reset_condition_val=100, succession_rate = 0.95)
sud.board

array([[9, 7, 0, 0, 0, 0, 4, 0, 0],
       [0, 0, 8, 0, 0, 6, 0, 3, 7],
       [0, 0, 0, 0, 0, 7, 0, 9, 5],
       [1, 2, 0, 0, 3, 5, 6, 0, 0],
       [0, 8, 0, 0, 0, 0, 0, 0, 1],
       [5, 0, 0, 0, 0, 2, 0, 4, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 4, 0, 0, 0, 5, 1, 2],
       [7, 5, 2, 0, 6, 0, 3, 8, 0]])

In [12]:
results = new_solver.solve(sudoku=sud, is_candidate_mode=True)

Improvement! Score: 92.01%
Epoch: 0 best_global: 92.01%, best_local: 92.01%
Improvement! Score: 92.83%
Epoch: 1 best_global: 92.83%, best_local: 92.83%
Epoch: 2 best_global: 92.83%, best_local: 92.66%
Improvement! Score: 93.42%
Epoch: 3 best_global: 93.42%, best_local: 93.42%
Epoch: 4 best_global: 93.42%, best_local: 93.2%
Improvement! Score: 93.78%
Epoch: 5 best_global: 93.78%, best_local: 93.78%
Epoch: 6 best_global: 93.78%, best_local: 93.49%
Epoch: 7 best_global: 93.78%, best_local: 93.54%
Epoch: 8 best_global: 93.78%, best_local: 93.26%
Epoch: 9 best_global: 93.78%, best_local: 93.58%
Epoch: 10 best_global: 93.78%, best_local: 93.34%
Epoch: 11 best_global: 93.78%, best_local: 93.33%
Epoch: 12 best_global: 93.78%, best_local: 92.66%
Epoch: 13 best_global: 93.78%, best_local: 93.5%
Improvement! Score: 93.85%
Epoch: 14 best_global: 93.85%, best_local: 93.85%
Epoch: 15 best_global: 93.85%, best_local: 93.08%
Epoch: 16 best_global: 93.85%, best_local: 93.31%
Epoch: 17 best_global: 93.8

In [13]:
best_chrom, score_best_global, best_per_iter, history = results

In [14]:
best_chrom

array([[9, 7, 5, 8, 5, 1, 4, 6, 6],
       [2, 4, 8, 9, 2, 6, 1, 3, 7],
       [6, 3, 1, 4, 8, 7, 8, 9, 5],
       [1, 2, 7, 8, 3, 5, 6, 7, 9],
       [4, 8, 9, 6, 7, 9, 2, 5, 1],
       [5, 6, 7, 1, 9, 2, 9, 4, 8],
       [8, 1, 3, 3, 2, 4, 7, 6, 9],
       [6, 9, 4, 7, 8, 9, 5, 1, 2],
       [7, 5, 2, 9, 6, 9, 3, 8, 4]])